# **SIC Capstone Project**

*Supermarket Project*

### **Abstract Data Structures**

In [10]:
class Queue:
    
    def __init__(self):
        self.queue = []

    def __str__(self):
        return "{}".format(self.queue)

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        return None if self.is_empty() else self.queue.pop(0)

    def peek_first(self):
        return None if self.is_empty() else self.queue[0]

    def peek_last(self):
        return None if self.is_empty() else self.queue[-1]

### **Custom Exceptions**

In [11]:
class ClientNotFoundError(Exception):
    '''Triggers when the client_name is not in the agents dictionary'''
    def __init__(self, client_name):
        self.message = f"❌ Error: Client: '{client_name.capitalize()}' not found"
        super().__init__(self.message)

class SupermarketNotFoundError(Exception):
    '''Triggers when the supermarket_name is not in the agents dictionary'''
    def __init__(self, supermarket_name):
        self.message = f"❌ Error: Supermarket: '{supermarket_name.capitalize()}' not found"
        super().__init__(self.message)

class ClientNotInSupermarketError(Exception):
    '''Triggers when the client is not in a supermarket'''
    def __init__(self, client_name):
        self.message = f"⚠️ Client '{client_name.capitalize()}' is not in a supermarket at this moment"
        super().__init__(self.message)

class ClientAlreadyInASupermarketError(Exception):
    '''Triggers when the client is already in a supermarket'''
    def __init__(self, client_name, supermarket_name):
        self.message = f"⚠️ Client: '{client_name.capitalize()}' is already in Supermarket: ({supermarket_name.capitalize()}) at this moment.",
        super().__init__(self.message)

class SupermarketClosedError(Exception):
    '''Triggers when the supermarket is closed when trying to execute some commands'''
    def __init__(self, supermarket_name):
        self.message = f"⚠️ Supermarket {supermarket_name} is closed at this moment."
        super().__init__(self.message)

class ClientNotFirstInLineError(Exception):
    '''Triggers when the client is not the first at the checkout line'''
    def __init__(self, client_name):
        self.message = f"⚠️ Can't perform this action. Client '{client_name.capitalize()}' is not first in line at this moment."
        super().__init__(self.message)

class ClientInCheckoutLineError(Exception):
    '''Triggers when the client is at the checkout line and is trying to execute certain commands'''
    def __init__(self, client_name):
        self.message = f"⚠️ Can't perform this action. Client '{client_name.capitalize()}' is at the checkout line at this moment"
        super().__init__(self.message)

class ProductNotInInventoryError(Exception):
    '''Triggers when the specified product is not in the supermarket inventory'''
    def __init__(self, product_name, supermarket_name):
        self.message = f"⚠️ Can't perform this action. {product_name.capitalize()} is not in {supermarket_name.capitalize()}'s inventory"
        super().__init__(self.message)

class ProductNotEnoughStockError(Exception):
    '''Triggers when the specified product doesn't have enough stock for the client to buy'''
    def __init__(self, supermarket_name, product_name, product_stock):
        self.message = f"⚠️ Can't perform this action. Not enough stock of {product_name.capitalize()} in {supermarket_name.capitalize()}'s inventory. Current stock: {product_stock}"
        super().__init__(self.message)

class ProductNotInCartError(Exception):
    '''Triggers when the specified product is not in the client's cart'''
    def __init__(self, client_name, product_name):
        self.message = f"⚠️ '{product_name.capitalize()} NOT in {client_name.capitalize()}'s cart"
        super().__init__(self.message)

class InvalidRemovalAmountError(Exception):
    '''Triggers when the amount of product to remove exceeds the current amount in the client's cart'''
    def __init__(self, client_name, product_name, current_amount):
        self.message = f"⚠️ Can't remove product. {client_name.capitalize()}'s cart only has {current_amount} of '{product_name.capitalize()}'"
        super().__init__(self.message)

class ClientEmptyCartError(Exception):
    '''Triggers when the client tries to get in the checkout line and their cart is empty'''
    def __init__(self, client_name):
        self.message = f"⚠️ Can't get in line. {client_name.capitalize()}'s cart is empty'"
        super().__init__(self.message)

class ClientHasNotPaidError(Exception):
    '''Triggers when the client tries to leave without paying'''
    def __init__(self, client_name):
        self.message = f"⚠️ Can't leave the supermarket yet. {client_name} has not paid for the products in their cart"
        super().__init__(self.message)

### **Class Hierarchy**

#### Agent

In [12]:
class Agent:
    """Clase base para todos los agentes en el sistema."""
    
    def __init__(self, name):
        self.name = name

    def describe(self):
        return f"Agent: '{self.name.capitalize()}'"

#### Client

In [13]:
from random import uniform
import json, copy

class Client(Agent):
    """Clase que representa a un cliente que interactúa con el supermercado."""

    def __init__(self, name):
        super().__init__(name)
        self.status  = { 
            "is_in_supermarket": False, 
            "supermarket": "", 
            "has_paid": False, 
            "wallet": round(uniform(1000.0, 5000.0), 2),
            "is_checking_out": False,
            "is_in_checkout_line": False
        }
        self.shopping_cart = {}

    def __str__(self):
        return f'Client: "{self.name.capitalize()}"'

    def move_to_supermarket(self, supermarket_name):
        self.status["is_in_supermarket"] = True
        self.status["supermarket"] = supermarket_name

    def pick_up_product(self, product_name, amount): # Al comprar un producto, debemos actualizar el inventario??
        if product_name in self.shopping_cart.keys():
            self.shopping_cart[product_name] += amount
        else:
            self.shopping_cart.update({ product_name: amount })

    def show_cart(self):
        if not self.shopping_cart:
            print(f"⚠️ No products in shopping cart yet.")
        else: 
            print(f"🛒 {self.name.capitalize()}'s cart:")
            for product, amount in self.shopping_cart.items():
                print(f'\t 📌 "{product.capitalize()}" : {amount} units.')
    
    def remove_from_cart(self, product_name, amount):
        if self.shopping_cart[product_name] > amount:
            self.shopping_cart[product_name] -= amount
        elif self.shopping_cart[product_name] == amount:
            del self.shopping_cart[product_name]

    def leave_line(self):
        self.status["is_in_checkout_line"] = False

    def leave_supermarket(self):
        self.status["has_paid"] = False
        self.status["is_in_supermarket"] = False
        self.status["supermarket"] = ""
    
    def get_in_line(self):    
        self.status["is_in_checkout_line"] = True

    def is_in_supermarket(self):
        return self.status["is_in_supermarket"]
    
    def is_in_checkout_line(self):
        return self.status["is_in_checkout_line"]
    
    def is_product_in_cart(self, product_name):
        return product_name in self.shopping_cart.keys()

    def info(self):
        return f'''
            Client name: {self.name.capitalize()},
            Status: { json.dumps(self.status) },
            shopping_cart: { json.dumps(self.shopping_cart) },
        '''        

#### Supermarket

In [14]:
import random

class Supermarket(Agent):
    """Clase que representa un supermercado"""

    def __init__(self, name):
        super().__init__(name)
        self.status = {
            "clients": [], 
            "checkout_line": Queue(),
            "is_open": True
        }
        self.inventory = {}
        
    def info(self):
        return f'''
            Supermarket name: {self.name.capitalize()}
            Status: {json.dumps(self.status)}
            Inventory: { json.dumps(self.inventory) }
        '''

    def add_product(self, product_name):
        """ Añade un nuevo producto al supermercado """
        product_name = product_name
        if product_name in self.inventory:                          #comprobamos si el producto se encuentra en el supermercado
            print(f'❌ Error: "{product_name.capitalize()}" already exist in "{self.name.capitalize()}".')
        else:
            product_price = round(random.uniform(1.0, 20.0), 2)    # Generamos un precio aleatorio entre 1.00 y 20.00€
            product_stock = random.randint(1, 20)                  # Stock inicial con numero random
            self.inventory[product_name] = {
                "price" : product_price,
                "stock" : product_stock,
            }
            print(f'✔️ Product: "{product_name.capitalize()}" added to "{self.name.capitalize()}" with price: {product_price}€ and stock: {product_stock} units.')  
    
    def show_inventory(self):
        """ Muestra todo el inventario del supermercado """
        if not self.inventory:
            print(f'⚠️ There are no products in {self.name}.')
        else: 
            print('📑 SUPERMARKET INVENTORY :')
            for product, details in self.inventory.items():
                price = details['price']
                stock = details['stock']
                print(f'\t 📌 Product: "{product.capitalize()}" | Price: {price}€ | Stock: {stock} units.')
    
    def show_products(self):
        """ Muestra la lista de productos del supermercado """
        if not self.inventory:
            print(f'⚠️ There are no products in "{self.name.capitalize()}".')
        else: 
            print('📑 LIST OF PRODUCTS :')
            for product in self.inventory.keys():
                print(f'\t 📌 {product.capitalize()}')

    def restock_product(self, product_name, quantity): 
        """ Reabastecer un producto en el supermercado """
        if not product_name in self.inventory:
            print(f'❌ Error: Product: "{product_name.capitalize()}" not found in "{self.name.capitalize()}"')
        elif not int(quantity) or int(quantity) < 1:               # Comprobamos que se introduce un numero positivo
            print('❌ Error: Please, enter a positive integer to restock the product.')
        else: 
            self.inventory[product_name]['stock'] += int(quantity) # Sumamos a la cantidad actual el restock
            new_stock = self.inventory[product_name]['stock']
            print(f'✔️ Product: "{product_name.capitalize()}" restocked. New Stock: {new_stock} units.')
    
    def set_open_close(self, state): 
        """ Abrir/Cerrar el supermercado si no hay clientes dentro """
        if state == 'close':
            if not self.status['clients']:
                self.status['is_open'] = False
                print(f'✔️ Supermarket "{self.name.capitalize()}" closed.')
            else: 
                print(f'❌ Error: Supermarket "{self.name.capitalize()}" can not be closed because there are clients inside.')

        elif state == 'open':
            if self.status['is_open']:
                print(f'⚠️ Supermarket "{self.name.capitalize()}" is already open.')
            else: 
                self.status['is_open'] = True
                print(f'✔️ Supermarket "{self.name.capitalize()}" open.')

    def show_clients(self):
        """ Muestra la lista de clientes en el supermercado """
        if not self.status['clients']:
            print(f'⚠️ There are no clients in "{self.name.capitalize()}".')
        else:
            print('🙋‍♂️ LIST OF CLIENTS :')
            for client in self.status['clients']:
                print(f'\t 📌 {client}') 
    
    def show_clients_in_cart(self):
        """ Mostrar la lista de clientes en la caja """
        if not self.status['checkout_line'].queue:
            print('⚠️ There are no clients in line yet.')
        else:
            print('🛒 CLIENTS IN CART')
            for client in self.status['checkout_line'].queue:
                print(f'\t 📌 {client}')

    def serve_next_customer(self):
        """ Atender al siguiente cliente en la caja """
        if not self.status['checkout_line'].is_empty:           # Comprobamos si quedan clientes en la cola
            print('⚠️ The are no clients in line.')
        else:
            client = self.status['checkout_line'].peek_first()     # Muestra el primero de la cola
            print(f'Attending to client: "{client}"')


#### CitySimulation

In [15]:
class CitySimulation:
    """Clase principal para gestionar la simulación de la ciudad."""

    def add_agent(self, agent_type, agent_name):                # Comprobamos que tanto cliente o supermercado no existan ya
        """Añade un nuevo agente al sistema."""
        if agent_type == 'client':
            if agent_name in agents:
                print(f'❌ Error: "{agent_name.capitalize()}" already exists in the System.')
            else:
                agents[agent_name] = Client(agent_name)
                print(f'✔️ {agent_type.capitalize()} : "{agent_name.capitalize()}" added to the System.')

        elif agent_type == 'supermarket':
            if agent_name in agents:
                print(f'❌ Error: {agent_name.capitalize()} already exists in the System.')
            else:
                agents[agent_name] = Supermarket(agent_name)
                print(f'✔️ {agent_type.capitalize()} : "{agent_name.capitalize()}" added to the System.')

    def remove_agent(self, agent_name):
        """Elimina un agente del sistema."""
        if agent_name in agents:
            del agents[agent_name]
            print(f'✔️ Agent "{agent_name.capitalize()}" removed from the System.')
        else:
            print(f'❌ Error: Agent "{agent_name.capitalize()}" not found.')

    def list_agents(self):
        """Muestra todos los agentes en el sistema."""
        global agents
        
        if not agents:
            print("⚠️ No agents in the System.")
        else: 
            print("CURRENT AGENTS: ")
            for agent in agents.values():
                print(agent.describe())

    def validate_move_command(self, client_name, supermarket_name):
        """Realiza todas las validaciones necesarias para el comando 'move'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.supermarket_exists(supermarket_name):
            raise SupermarketNotFoundError(supermarket_name)
        if self.is_supermarket_closed(supermarket_name):
            raise SupermarketClosedError(supermarket_name)
        if self.is_client_in_supermarket(client_name):
            raise ClientAlreadyInASupermarketError(client_name, agents[client_name].status["supermarket"])

    def move(self, client_name, supermarket_name):
        """Mover a un cliente a un supermercado específico"""
        client = agents[client_name]
        supermarket = agents[supermarket_name]
        client.move_to_supermarket(supermarket_name)
        supermarket.status["clients"].append(client) #### crear método en la clase Supermarket??

    def validate_pick_up_command(self, client_name, product_name, amount):
        """Realiza todas las validaciones necesarias para el comando 'pick_up'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)
        if self.is_client_in_checkout_line(client_name):
            raise ClientInCheckoutLineError(client_name)
        if not self.product_in_stock(product_name, agents[client_name].status["supermarket"]):
            raise ProductNotInInventoryError(product_name, agents[client_name].status["supermarket"])
        if self.not_enough_stock(agents[client_name].status["supermarket"], product_name, amount):
            supermarket = agents[agents[client_name].status["supermarket"]]
            product_stock = supermarket.inventory[product_name]["stock"]
            raise ProductNotEnoughStockError(agents[client_name].status["supermarket"], product_name, product_stock)

    def pick_up(self, client_name, product_name, amount):
        """Permitir que un cliente recoja un producto y lo añada a la cesta"""
        client = agents[client_name]
        client.pick_up_product(product_name, int(amount))
        supermarket = agents[client.status["supermarket"]]
        supermarket.inventory[product_name]["stock"] -= int(amount) #### crear método en la clase Supermarket??

    def validate_pay_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'pay'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)
        if not self.is_client_first_in_line(client_name):
            raise ClientNotFirstInLineError(client_name)

    def pay(self, client_name):
        """Pagar en caja todos los productos de la cesta"""
        client = agents[client_name]
        wallet = client.status["wallet"]
        cart = client.shopping_cart

        supermarket = agents[client.status["supermarket"]]
        inventory = supermarket.inventory

        total = 0
        for product, amount in cart.items():
            total += inventory[product]["price"] * amount
        if total <= wallet:
            wallet -= total
            cart.clear()
            client.status["has_paid"] = True
            #supermarket.status["checkout_line"].dequeue()
            print(f"✔️ '{client_name.capitalize()}' has paid for the items in their cart. Total price: {total}€")
        else:
            print(f"⚠️ '{client_name.capitalize()}' doesn't have enough money to pay for the products. Total price: {total} --- {client_name}'s wallet: {wallet}.")
            print("⚠️ Client must remove products from their cart until they can afford them.")

    def validate_show_cart_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'show_cart'."""
        if not self.client_exists(client_name):
            raise  ClientNotFoundError(client_name)                   
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)

    def show_cart(self, client_name):
        """Mostrar los productos en la cesta del cliente"""
        client = agents[client_name]
        client.show_cart()

    def validate_exit_line_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'exit_line'."""
        if not self.client_exists(client_name):
            raise  ClientNotFoundError(client_name)                   
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name) 
        if not self.is_client_first_in_line(client_name):
            raise ClientNotFirstInLineError(client_name)

    def exit_line(self, client_name):
        """Permitir al cliente salir de la cola siempre y cuando sea el primero"""
        client = agents[client_name]
        supermarket = agents[client.status["supermarket"]]
        client.leave_line()
        supermarket.status["checkout_line"].dequeue() #### crear método en la clase Supermarket??

    def validate_leave_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'leave'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)
        if not self.is_client_first_in_line(client_name):
            raise ClientNotFirstInLineError(client_name)
        if not self.has_client_paid(client_name):
            raise ClientHasNotPaidError(client_name)
    
    def leave(self, client_name):
        """Permite al cliente salir del supermerado después de haber pagado en caja"""
        client = client = agents[client_name]
        supermarket = agents[client.status["supermarket"]]
        client.leave_supermarket()
        supermarket.status["checkout_line"].dequeue()
        supermarket.status["clients"].remove(client) #### crear método en la clase Supermarket??

    def validate_leave_without_checkout_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'leave_without_checkout'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)
        if self.is_client_in_checkout_line(client_name):
            raise ClientInCheckoutLineError(client_name)
        if agents[client_name].shopping_cart:
            raise ClientHasNotPaidError()
        
    def leave_without_checkout(self, client_name):
        """Permite al cliente salir del supermercado sin pasar por caja"""
        client = client = agents[client_name]
        supermarket = agents[client.status["supermarket"]]
        client.leave_supermarket()
        supermarket.status["clients"].remove(client) #### crear método en la clase Supermarket??

    def validate_check_cart_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'check_cart'."""
        if not self.client_exists(client_name):
            raise  ClientNotFoundError(client_name)                   
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)

    def check_cart(self, client_name, product_name):
        """Comprueba si el producto se encuentra en la cesta del cliente"""
        client = agents[client_name]
        if client.is_product_in_cart(product_name):
            print(f"✔️ '{product_name.capitalize()}' is in {client_name.capitalize()}'s cart. Current amount: {client.shopping_cart[product_name]}")
        else:
            print(f"⚠️ '{product_name.capitalize()}' is NOT in {client_name.capitalize()}'s cart.")

    def validate_remove_from_cart_command(self, client_name, product_name, amount):
        """Realiza todas las validaciones necesarias para el comando 'remove_from_cart'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)
        if not self.is_product_in_cart(client_name, product_name):
            raise ProductNotInCartError(client_name, product_name)
        if self.amount_exceeded(client_name, product_name, amount):
            raise InvalidRemovalAmountError(client_name, product_name, agents[client_name].shopping_cart[product_name])       

    def remove_from_cart(self, client_name, product_name, amount):
        """Retira un producto de la cesta del cliente"""
        client = agents[client_name]
        supermarket = agents[client.status["supermarket"]]
        client.remove_from_cart(product_name, amount)
        supermarket.inventory[product_name]["stock"] += amount #### crear método en la clase Supermarket??
        print(f"✔️ {amount} of '{product_name.capitalize()}' removed from {client_name.capitalize()}'s cart.")

    def validate_get_in_line_command(self, client_name):
        """Realiza todas las validaciones necesarias para el comando 'get_in_line'."""
        if not self.client_exists(client_name):
            raise ClientNotFoundError(client_name)
        if not self.is_client_in_supermarket(client_name):
            raise ClientNotInSupermarketError(client_name)
        if self.is_client_in_checkout_line(client_name):
            raise ClientInCheckoutLineError(client_name)
        if not agents[client_name].shopping_cart:
            raise ClientEmptyCartError(client_name)       

    def get_in_line(self, client_name):
        """Permitir que el cliente se ponga en la fila para pagar"""
        client = agents[client_name]
        supermarket = agents[client.status["supermarket"]]
        client.get_in_line()
        supermarket.status["checkout_line"].enqueue(client) #### crear método en la clase Supermarket??

    def client_exists(self, client_name):
        """Comprueba si el usuario existe en el diccionario de agentes"""
        return client_name in agents and isinstance(agents[client_name], Client)
    
    def supermarket_exists(self, supermarket_name):
        """Comprueba si el supermercado existe en el diccionario de agentes"""
        return supermarket_name in agents and isinstance(agents[supermarket_name], Supermarket)
    
    def is_client_in_supermarket(self, client_name):
        """Comprueba si el cliente se encuentra en un supermercado"""
        return agents[client_name].is_in_supermarket()
    
    def is_client_in_checkout_line(self, client_name):
        """Comprueba si el cliente se encuentra en la cola"""
        return agents[client_name].is_in_checkout_line()
    
    def is_client_first_in_line(self, client_name):
        """Comprueba si el cliente es el primero de la cola"""
        supermarket = agents[agents[client_name].status["supermarket"]]
        if not supermarket.status["checkout_line"].peek_first():
            return False
        else:
            return client_name == supermarket.status["checkout_line"].peek_first().name
    
    def is_supermarket_closed(self, supermarket_name):
        """Comprueba si el supermercado está cerrado"""
        return agents[supermarket_name].status["is_open"] == False
    
    def is_product_in_cart(self, client_name, product_name):
        """Comprueba si el producto especificado se encuentra en el carrito del cliente"""
        return product_name in agents[client_name].shopping_cart.keys()
    
    def amount_exceeded(self, client_name, product_name, amount):
        """Comprueba si la cantidad de producto que se quiere eliminar del carrito excede la cantidad que hay en el carrito"""
        return agents[client_name].shopping_cart[product_name] < amount
    
    def product_in_stock(self, product_name, supermarket_name):
        """Comprueba si el producto se encuentra en el inventario del supermercado"""
        return product_name in agents[supermarket_name].inventory.keys()
    
    def not_enough_stock(self, supermarket_name, product_name, amount):
        """Comprueba si no hay suficiente stock de un producto"""
        return agents[supermarket_name].inventory[product_name]["stock"] < int(amount)
    
    def has_client_paid(self, client_name):
        return agents[client_name].status["has_paid"]

    def help(self):
        """Muestra la lista de comandos disponibles."""
        print("""
            Available commands:
            - supermarket add <supermarket_name>: Agregar un nuevo supermercado al sistema.
            - supermarket add_product <supermarket_name> <product_name>: Agregar un producto al inventario del supermercado.
            - client add <client_name>: Agregar un cliente al sistema.
            - client move <client_name> <to_supermarket>: Mover a un cliente a un supermercado específico.
            - client pick_up <client_name> <product_name>: Permitir que un cliente recoja un producto.
            - supermarket show_clients <supermarket_name>: Mostrar la lista de clientes en el supermercado.
            - supermarket show_products <supermarket_name>: Mostrar la lista de productos en el supermercado.
            - supermarket restock <supermarket_name> <product_name> : Reabastecer un producto en el supermercado.
            - client buy <client_name> <product_name>: Comprar un producto y agregarlo a la cesta del cliente.
            - client show_cart <client_name>: Mostrar los productos en la cesta del cliente.
            - client checkout <client_name>: Preparar al cliente para pagar en caja.
            - supermarket show_clients_in_cash <supermarket_name>: Mostrar la lista de clientes en la caja.
            - supermarket serve_next_customer <supermarket_name>: Atender al siguiente cliente en la fila de caja.
            - client leave <client_name>: Permitir que un cliente salga del supermercado, siempre que haya pagado.
            - client leave_without_checkout <client_name>: Permitir que un cliente salga sin comprar, si su cesta está vacía.
            - client check_cart <client_name> <product_name>: Verificar si un producto está en la cesta del cliente.
            - supermarket set_open_close <supermarket_name> <open/close>: Abrir / Cerrar el supermercado si no hay clientes dentro.
            - supermarket show_inventory <supermarket_name>: Mostrar todo el inventario del supermercado.
            - client remove_from_cart <client_name> <product_name>: Permitir que el cliente retire un producto de su cesta.
            - client get_in_line <client_name>: Permitir que un cliente se ponga en la fila para pagar.
            """)            

    def command_loop(self):
        """Bucle principal para gestionar comandos del usuario."""
        print("Starting city simulation 🏙️🏙️🏙️ Type 'q' to exit ore '?' for help!!")
        while True:
            command = input('> ')
            if command == 'q':
                print('See you next time! 👋')
                break
            self.process_command(command)

    def process_command(self, command):
        """Procesa los comandos ingresados por el usuario."""
        parts = command.split()
        if not parts:
            return
        
        cmd = parts[0]
        
        if cmd == '?':
            self.help()  # Llama al método de ayuda
            return
        
        if cmd == 'list':     # Comando para listar los agentes
            self.list_agents()
            return
        
        # supermarket commands
        elif cmd == 'supermarket':
            
            if parts[1] == 'add':
                try:
                    _, _, supermarket_name = parts
                    self.add_agent("supermarket", supermarket_name)
                except ValueError:
                    print("❌ Error: Invalid add command format. Use 'supermarket add <supermarket_name>'")

            elif parts[1] == 'add_product':
                try:
                    _, _, supermarket_name, product_name  = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].add_product(product_name)
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the System.')
                except ValueError:
                    print("❌ Error: Invalid add_product command format. Use 'supermarket add_product <supermarket_name> <product_name>'")

            elif parts[1] == 'show_clients':
                try:
                    _, _, supermarket_name = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].show_clients()
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the System.')
                except ValueError:
                    print("❌ Error: Invalid show_clients command format. Use 'supermarket show_clients <supermarket_name>'")

            elif parts[1] == 'show_products':
                try:
                    _, _, supermarket_name = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].show_products()
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the system.')
                except ValueError:
                    print("❌ Error: Invalid show_products command format. Use 'supermarket show_products <supermarket_name>'")

            elif parts[1] == 'restock':
                try:
                    _, _, supermarket_name, product_name, quantity  = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].restock_product(product_name, quantity)
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the system.')
                except ValueError:
                    print("❌ Error: Invalid add_product command format. Use 'supermarket restock <supermarket_name> <product_name> <quantity>'")

            elif parts[1] == 'show_clients_in_cart':
                try:
                    _, _, supermarket_name = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].show_clients_in_cart()
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the system.')
                except ValueError:
                    print("❌ Error: Invalid show_products command format. Use 'supermarket show_clients_in_cart <supermarket_name>'")

            elif parts[1] == 'serve_next_customer':
                try:
                    _, _, supermarket_name = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].serve_next_customer()
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the system.')
                except ValueError:
                    print("❌ Error: Invalid show_products command format. Use 'supermarket serve_next_customer <supermarket_name>'")

            elif parts[1] == 'set_open_close':
                try:
                    _, _, supermarket_name, open_close = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].set_open_close(open_close)
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the system.')
                except ValueError:
                    print("❌ Error: Invalid show_products command format. Use 'supermarket set_open_close <supermarket_name <open/close>'")

            elif parts[1] == 'show_inventory':
                try:
                    _, _, supermarket_name = parts
                    if supermarket_name in agents:
                        agents[supermarket_name].show_inventory()
                    else: 
                        print(f'❌ Error: "{supermarket_name.capitalize()}" not found in the system.')
                except ValueError:
                    print("❌ Error: Invalid show_products command format. Use 'supermarket show_inventory <supermarket_name>'")


        # client commands
        elif cmd == 'client':

            if parts[1] == 'add':
                try:
                    _, _, client_name = parts
                    self.add_agent('client', client_name)
                except ValueError:
                    print("❌ Error: Invalid add command format. Use 'client add <client_name>'")

            elif parts[1] == 'remove_client':
                try:
                    _, _, client_name = parts
                    self.remove_agent(client_name)
                except ValueError:
                    print("❌ Error: Invalid remove_client command format. Use 'client remove_client <client_name>'")
            
            elif parts[1] == 'move':
                try:
                    _, _, client_name, supermarket_name = parts
                    self.validate_move_command(client_name, supermarket_name)                  
                except ValueError:
                    print("❌ Error: Invalid move command format. Use 'client move <client_name> <supermarket_name>'")
                except (ClientNotFoundError, SupermarketNotFoundError, SupermarketClosedError, ClientAlreadyInASupermarketError) as e:
                    print(e)
                else:
                    self.move(client_name, supermarket_name)
                    print(f"✔️ Client: {client_name} moved to {supermarket_name}.")

            elif parts[1] == 'pick_up':
                try:
                    _, _, client_name, product_name, amount = parts
                    self.validate_pick_up_command(client_name, product_name, amount)
                except ValueError:
                    print("❌ Error: Invalid pick_up command format. Use 'client pick_up <client_name> <product_name> <amount>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ClientInCheckoutLineError, ProductNotInInventoryError, ProductNotEnoughStockError) as e:
                    print(e) 
                else:
                    self.pick_up(client_name, product_name, amount)
                    print(f"✔️ {amount} of '{product_name}' added to {client_name}'s cart.")

            elif parts[1] == 'pay':
                try:
                    _, _, client_name = parts
                    self.validate_pay_command(client_name)
                except ValueError:
                    print("❌ Error: Invalid pay command format. Use 'client pay <client_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ClientNotFirstInLineError) as e:
                    print(e)
                else:
                    self.pay(client_name)

            elif parts[1] == 'show_cart':
                try:
                    _, _, client_name = parts
                    self.validate_show_cart_command(client_name)                   
                except ValueError:
                    print("❌ Error: Invalid show_cart command format. Use 'client show_cart <client_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError) as e:
                    print(e)                       
                else:
                    self.show_cart(client_name)

            elif parts[1] == 'exit_line':
                try:
                    _, _, client_name = parts
                    self.validate_exit_line_command(client_name)                  
                except ValueError:
                    print("❌ Error: Invalid exit_line command format. Use 'client exit_line <client_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ClientNotFirstInLineError) as e:
                    print(e)           
                else:
                    self.exit_line(client_name)
                    print(f"✔️ {client_name.capitalize()} has left the checkout line")

            elif parts[1] == 'leave':
                try:
                    _, _, client_name = parts
                    self.validate_leave_command(client_name)
                except ValueError:
                    print("❌ Error: Invalid leave command format. Use 'client leave <client_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ClientNotFirstInLineError, ClientHasNotPaidError) as e:
                    print(e)
                else:
                    self.leave(client_name)
                    print(f"✔️ {client_name.capitalize()} has left the supermarket")

            elif parts[1] == 'leave_without_checkout':
                try:
                    _, _, client_name = parts
                    self.validate_leave_without_checkout_command(client_name)
                except ValueError:
                    print("❌ Error: Invalid leave_without_checkout command format. Use 'client leave_without_checkout <client_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ClientInCheckoutLineError, ClientHasNotPaidError) as e:
                    print(e)
                else:
                    self.leave_without_checkout(client_name)
                    print(f"✔️ {client_name.capitalize()} has left the supermarket.")

            elif parts[1] == 'check_cart':
                try:
                    _, _, client_name, product_name = parts
                    self.validate_check_cart_command(client_name, product_name)
                except ValueError:
                    print("❌ Error: Invalid check_cart command format. Use 'client check_cart <client_name> <product_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError) as e:
                    print(e)
                else:
                    self.check_cart(client_name, product_name)

            elif parts[1] == 'remove_from_cart':
                try:
                    _, _, client_name, product_name, amount = parts
                    amount = int(amount)
                    self.validate_remove_from_cart_command(client_name, product_name, amount)
                except ValueError:
                    print("❌ Error: Invalid remove_from_cart command format. Use 'client remove_from_cart <client_name> <product_name> <amount>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ProductNotInCartError, InvalidRemovalAmountError) as e:
                    print(e)                 
                else:
                    self.remove_from_cart(client_name, product_name, amount)

            elif parts[1] == 'get_in_line':
                try:
                    _, _, client_name = parts
                    self.validate_get_in_line_command(client_name)
                except ValueError:
                    print("❌ Error: Invalid get_in_line command format. Use 'client get_in_line <client_name>'")
                except (ClientNotFoundError, ClientNotInSupermarketError, ClientInCheckoutLineError, ClientEmptyCartError) as e:
                    print(e)
                else:
                    self.get_in_line(client_name)
                    print(f"✔️ {client_name.capitalize()} is now last in the checkout line.")

        else:
            print("❌ Error: Unknown command. Type 'help' for a list of commands.")

### **Diccionario global para almacenar agentes**

In [16]:
agents = {}

### **Main program**

In [17]:
if __name__ == "__main__":
    simulation = CitySimulation()
    simulation.command_loop()

Starting city simulation 🏙️🏙️🏙️ Type 'q' to exit ore '?' for help!!
✔️ Client : "Andrei" added to the System.
✔️ Supermarket : "Mercadona" added to the System.
✔️ Client: Andrei moved to Mercadona.
❌ Error: Unknown command. Type 'help' for a list of commands.


AttributeError: 'NoneType' object has no attribute 'name'